In [1]:
# (2026.02.01)
# Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
# 체인을 수동으로 구현해야 합니다.
# 체인에 ConversationBufferMemory를 부여합니다.
# 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
# 체인에 다음 질문을 합니다:
# Aaronson 은 유죄인가요?
# 그가 테이블에 어떤 메시지를 썼나요?
# Julia 는 누구인가요?

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import  UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings,CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate


memory = ConversationBufferMemory(
    return_messages=True, 
    memory_key="history"
)


llm = ChatOpenAI(temperature=0.1,verbose=True)
cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(  #AI model 이 이해하는 토큰 방식을 사용해서 
    separator="\n\n",
    chunk_size = 600,
    chunk_overlap = 50,

)


loader = UnstructuredFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)

embedder = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embedder,cache_dir,
)

vectorstores = FAISS.from_documents(docs,cached_embeddings) 


retriever = vectorstores.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer question using only the following context.If you know know the answer just say you don't know. Don't make it up :\n\n{context} "),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}")
]
)


chain = prompt | llm 

questions = [
    "Aaronson 은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?",
    "Julia 는 누구인가요?"
]


for question in questions:

    searched_docs = retriever.invoke(question)
    
    context_text = "\n\n".join(doc.page_content for doc in searched_docs)
    
    history_data = memory.load_memory_variables({})["history"]
    
    print(f"\n[Human]: {question}")
    
    response = chain.invoke({
        "context": context_text,
        "history": history_data,
        "question": question
    })
    
    print(f"[AI]: {response.content}")
    

    memory.save_context(
        {"input": question},
        {"output": response.content}
    )


[Human]: Aaronson 은 유죄인가요?
[AI]: Aaronson, Jones, 그리고 Rutherford는 그들이 혐의를 받았던 범죄의 유죄를 인정했습니다.

[Human]: 그가 테이블에 어떤 메시지를 썼나요?
[AI]: 그가 쓴 메시지는 "FREEDOM IS SLAVERY"와 "TWO AND TWO MAKE FIVE" 그리고 "GOD IS POWER" 입니다.

[Human]: Julia 는 누구인가요?
[AI]: Julia는 주인공인 윈스턴과 사랑을 나누었던 여성입니다.
